<a href="https://colab.research.google.com/github/Ayrsz/SignalAndSistemyProject/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
#ML
import sklearn as sk

#Scientific computation
import numpy as np
import jax
import jax.numpy as jnp

#Plot and view
from matplotlib import pyplot as plt
import IPython as ipy
from IPython import display
from IPython.display import Audio

#Data manipulation
import os
import gc
import pandas as pd
import cv2 as cv

#Audio manipulation
import librosa
import soundfile as sf


In [69]:
from zipfile import ZipFile
try:
    from google.colab import drive
    drive.mount("/content/drive", force_remount= True)
except Exception as e:
    print(e)
    print("Código rodando localmente")

path_root = "/home/mas11/Documents/Datasets"


No module named 'google.colab'
Código rodando localmente


# Dataset config

In [70]:
data = pd.read_csv(path_root + "/Data/feats.csv")
data.head()

,name,dir_image,rms_mean,rms_std,rms_max,rms_min,rms_25p,rms_50p,rms_75p,zero_cross_mean,...,mfcc#09_std,mfcc#10_mean,mfcc#10_std,mfcc#11_mean,mfcc#11_std,mfcc#12_mean,mfcc#12_std,tempo,kurtosis,label
0,classical.00521.wav,/home/mas11/Documents/Datasets/images/classica...,0.0146,0.0044,0.0267,0.0058,0.0110,0.0147,0.0176,0.0578,...,7.0562,-9.1809,7.6074,-0.9181,5.9257,-7.7351,6.7801,[103],0.0003,classical
1,classical.00270.wav,/home/mas11/Documents/Datasets/images/classica...,0.0327,0.0151,0.0654,0.0038,0.0202,0.0276,0.0493,0.0510,...,6.4792,-4.6224,7.2181,-10.0690,7.6384,-4.8233,7.2265,[123],0.0122,classical
2,classical.00232.wav,/home/mas11/Documents/Datasets/images/classica...,0.0285,0.0092,0.0526,0.0052,0.0209,0.0279,0.0356,0.0431,...,9.4247,-4.2221,7.9807,-1.1659,6.6576,0.4751,9.2323,0,0.0030,classical
3,classical.00014.wav,/home/mas11/Documents/Datasets/images/classica...,0.0334,0.0216,0.0832,0.0094,0.0171,0.0230,0.0485,0.0684,...,8.8088,-8.1923,8.0725,-0.8276,9.9700,0.9924,12.2557,[143],0.1818,classical
4,classical.00597.wav,/home/mas11/Documents/Datasets/images/classica...,0.0633,0.0283,0.1181,0.0158,0.0399,0.0539,0.0896,0.0533,...,6.9916,-10.4254,8.0424,-1.1135,9.4015,-4.6708,9.5551,[135],0.1320,classical


In [71]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5994 entries, 0 to 5993
Data columns (total 59 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     5994 non-null   object 
 1   dir_image                5994 non-null   object 
 2   rms_mean                 5994 non-null   float64
 3   rms_std                  5994 non-null   float64
 4   rms_max                  5994 non-null   float64
 5   rms_min                  5994 non-null   float64
 6   rms_25p                  5994 non-null   float64
 7   rms_50p                  5994 non-null   float64
 8   rms_75p                  5994 non-null   float64
 9   zero_cross_mean          5994 non-null   float64
 10  zero_cross_std           5994 non-null   float64
 11  zero_cross_max           5994 non-null   float64
 12  zero_cross_min           5994 non-null   float64
 13  zero_cross_25p           5994 non-null   float64
 14  zero_cross_50p          

# Treating data

In [72]:
unvetorize = lambda num: float(num.strip("[]"))


data["tempo"] = data["tempo"].apply(unvetorize)

#Alguns estão com tempo 0, vou resolver
data = data[data["tempo"] != 0]

In [73]:
change_dir = lambda path: path.replace('Datasets', 'Datasets/Data')
data["dir_image"] = data["dir_image"].apply(change_dir)

In [74]:
# Dicionario, label -> Idx (labelEnconder)
#Transformar classes em numeros
label_mapping = {label: idx for idx, label in enumerate(data["label"].unique())}
data["label"] = data["label"].map(label_mapping)

In [75]:
data = data.dropna()

# Analise exploratória

# Modelo

In [ ]:
non_feat = ["label", "dir_image", "name"]

columns = data.columns
data_feat = data.drop(columns = non_feat)
data_non_feat = data.drop(columns = data_feat.columns)

names = data["name"]
dir_imgs = data["dir_image"]
labels = data["label"]

labels.head()


0    0
1    0
3    0
4    0
5    0
Name: label, dtype: int64

In [77]:
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(data_feat, labels, test_size= 0.2)

In [87]:
model = sk.ensemble.RandomForestClassifier()
model.fit(X_train, Y_train)

RandomForestClassifier()

In [93]:
pred = model.predict(X_test)
print(f"Numero de erros  {np.sum(pred != Y_test)}")
print(f"Numero acertos, {np.sum(pred == Y_test)}")
print(f"Precisão : {np.sum(pred == Y_test)*100/len(pred)}%")


Numero de erros  378
Numero acertos, 816
Precisão : 68.34170854271356%
